# Zero-Shot-Classification Model (SAVE & UPLOAD)

In [ ]:
# # MOUNT DRIVE
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# # DOWNLOAD Zero-Shot-Classification MODEL Hugging Face Hub

# from transformers import AutoModelForSequenceClassification, AutoTokenizer

# model_name = "facebook/bart-large-mnli"
# save_path = "/content/drive/MyDrive/Colab Notebooks/Projects/Book Recommendation/venv_cache/zero_shot_classification"

# model = AutoModelForSequenceClassification.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# # # SAVE Zero-Shot-Classification MODEL TO DRIVE

# model.save_pretrained(save_path)
# tokenizer.save_pretrained(save_path)

In [6]:
# # LOADS Zero-Shot-Classification MODEL FROM DRIVE (Device: GPU=0, CPU=-1) (Check for GPU, !nvidia-smi)

from transformers import pipeline

model_path = "/content/drive/MyDrive/Colab Notebooks/Projects/Book Recommendation/venv_cache/zero_shot_classification"

classifier = pipeline(
    "zero-shot-classification",
    model=model_path,
    tokenizer=model_path,
    device=0
)

Device set to use cuda:0


# Predict Missing Book Categories

In [8]:
import pandas as pd
import numpy as np
from typing import List, Any
from tqdm.notebook import tqdm

In [9]:
books = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Projects/Book Recommendation/books/books_cleaned.csv")
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description,simple_categories
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,NaN
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...,NaN
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...,NaN
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...,NaN
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,NaN
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction


In [10]:
# # HELPER FUNCTIONS

# Generates a predicted category based sequence
def generate_prediction(classifier: Any, sequence: str, categories: List[str]) -> str:
  # Generates prediced category based on sequence (book description)
  prediction = classifier(sequence, categories)

  # Generates predicted labels and probability scores for each category
  labels = prediction.get("labels", [])
  scores = prediction.get("scores", [])

  # Check that label and score exists
  if not labels or not scores:
    raise ValueError("Empty prediction returned by classifier")

  # Check that label and score match
  if len(labels) != len(scores):
    raise ValueError("Labels and scores length mismatch")

  # Return most likely category based on highest probability score
  return max(zip(scores, labels))[1]

# Generates accuracy of predicition DataFrame (1 or 0 = correct or incorrect)
def generate_accuracy(predicted_df: pd.DataFrame) -> np.float64:
    # Check that the column exists
    if "correct_prediction" not in predicted_df:
        raise KeyError("'correct_prediction' column missing")

    # Handle empty DataFrame
    if len(predicted_df) == 0:
        return np.float64(0.0)

    # Compute accuracy
    return np.float64(predicted_df["correct_prediction"].sum() / len(predicted_df))

def generate_missing_categories(classifier: Any, categories: List[str]) -> pd.DataFrame:
  # bwmc = books with missing categories

  # Create list for isbns and predicted categories for each bwmc
  isbns = []
  predicted_categories = []

  # Get bwmc's
  missing_categories = books.loc[books["simple_categories"].isna(), ["isbn13", "description"]].reset_index(drop=True)

  # Generate best category from categories for bwmc
  for i in tqdm(range(len(missing_categories))):
    sequence = missing_categories["description"].iloc[i]
    predicted_categories += [generate_prediction(classifier, sequence, categories)]
    isbns += [missing_categories["isbn13"].iloc[i]]

  # Return df that stores isnb and predicted category for bwmc
  return pd.DataFrame({"isbn13": isbns, "predicted_categories": predicted_categories})

def merge_books_with_missing_categories(books: pd.DataFrame, missing_predicted_df: pd.DataFrame) -> pd.DataFrame:
  if "simple_categories" not in books:
    raise KeyError("simple_categories not found")

  if "isbn13" not in books or "isbn13" not in missing_predicted_df:
    raise KeyError("isbn13 in books or missing_categories not found")

  # Merge books and missing categories df
  books = pd.merge(books, missing_predicted_df, on="isbn13", how="left")

  # Fill missing simple categories with predicted category
  books["simple_categories"] = books["simple_categories"].fillna(books["predicted_categories"])

  # ieseturn a new df with complete simple category
  return books.drop(columns="predicted_categories")

In [11]:
# # SIMPLE CATEGORIES
fiction_categories = ["Fiction", "Nonfiction"]

In [17]:
# TEST MODEL BY PREDICTING AND COMPARING TO TRUE CATEGORY

# actual_categories = []
# predicted_categories = []

# for i in tqdm(range(0, 300)):
#   sequence = books.loc[books["simple_categories"]=="Fiction", "description"].reset_index(drop=True)[i]
#   predicted_categories += [generate_prediction(classifier, sequence, fiction_categories)]
#   actual_categories += ["Fiction"]

# for i in tqdm(range(0, 300)):
#   sequence = books.loc[books["simple_categories"]=="Nonfiction", "description"].reset_index(drop=True)[i]
#   predicted_categories += [generate_prediction(classifier, sequence, fiction_categories)]
#   actual_categories += ["Nonfiction"]

# predicted_df = pd.DataFrame({'actual_category': actual_categories, 'prediction_category': predicted_categories})
# predicted_df

# predicted_df["correct_prediction"] = (
#     np.where(predicted_df["actual_category"]==predicted_df["prediction_categories"], 1, 0)
# )

# get_accuracy(predicted_df)

In [12]:
missing_predicted_df = generate_missing_categories(classifier, fiction_categories)

  0%|          | 0/1578 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [13]:
missing_predicted_df

,isbn13,predicted_categories
0,9780002261982,Fiction
1,9780006280897,Nonfiction
2,9780006280934,Nonfiction
3,9780006380832,Nonfiction
4,9780006470229,Fiction
...,...,...
1573,9788125026600,Nonfiction
1574,9788171565641,Fiction
1575,9788172235222,Fiction
1576,9788173031014,Nonfiction


In [65]:
books = merge_books_with_missing_categories(books, missing_predicted_df)

In [ ]:
books

In [67]:
books["simple_categories"].value_counts()

,count
simple_categories,
Fiction,2863
Nonfiction,1887
Children's Fiction,390
Childrens's Nonfiction,57


In [69]:
books.to_csv("/content/drive/MyDrive/Colab Notebooks/Projects/Book Recommendation/books/books_simple_prediction.csv", index=False)